### Deep Learning in practice

#### In this activity I will build a simple cat neural network classifier with TensorFlow 2.0

* Build a neural network that classifies cat images
* Train the neural network
* and finally, evaluate the accuracy of the model

* **PIL** (Python Imaging Library) is the python Imaging library that adds support for opening, manipulating, and saving many different image file formats.
* **random** implements pseudo random number generators for various distributions
* **numpy**  for numpy array manipulation - every image must be embedded into a numpy before it can be used as input for a neural network
* **os** operating system module and allows access to folders and files stored in the hard drive - useful for navigating the file system
* **tensorflow.keras** tensorflow module that will be used to build our neural network model

In [9]:
#importing the required libraries
import tensorflow  as tf 
print(tf.__version__)

#import pillow
from PIL import Image
from random import shuffle, choice

#import numpy arrays library
import numpy as np

#import os module
import os
#import keras and its libs
from tensorflow.keras import datasets, layers, models

#stop the execution from hanging, add this line to run training without interruption
os.environ ['KMP_DUPLICATE_LIB_OK'] = 'True'

2.16.1


### One-Hot Encoding

One-hot encoding takes a string as a parameter, and creates a label to the image.  Note that the label for a cat is  the numpy array [1,0] and the label for no cat is [0,1].  This technique is very common in machine learning.  **One-hot encoding** is a process by which categorical variables from string format to numerical format i.e. are converted into a format that could be provided to an ML algorithm to do a better prediction job.

*In one-hot encoding you create a vector containing all zeros except in one location*

The labels in the training data need to be one-hot encoded!

Think of a one-hot coded array [1,0] being the probability in this example of being or not being a cat

In [12]:
IMAGE_SIZE = 256 # we will be resizing the images to 256x256 pixels, this is a changeable hyperparameter
#increasing image size = longer training time but better accuracy
def one_hot_encode(className):
  '''One-hot encodes a given class'''
  if className == 'cats':return np.array([1,0])
  elif className =='nocats': return np.array([0,1])

In [13]:
print(f'cats: {one_hot_encode('cats')}')
print(f'cats: {one_hot_encode('nocats')}')

cats: [1 0]
cats: [0 1]


#### Loading the training data
The function **load_data** is responsible for loading the dataset.  First we set the IMAGE_DIRECTORY variable to point at the training dataset 'datasets/training_set'

The training data will be loaded into the list **train_data**.  Initially the list is empty.  Then the variable directories will get the tree structure of all directories in the specified path.

**os.walk()** generates the filenames in the directory tree by walking the tree either top-down or bottom-up.  For each directory in the tree rooted at directory top (including itself), it yields a 3-tuple (dirpath, dirnames, filenames)
* root: Prints out directories only from what we have specified
* dirs: Prints out sub-directories from the root
* files: Prints out all files from root and directories.

In [14]:
IMAGE_DIRECTORY = 'datasets/training_set'
def load_data (IMAGE_DIRECTORY, number_of_images = 100, shuffle_data = True):
  '''Loads the required image data'''
  print('loading images...')
  train_data =[]
  folders = next(os.walk(IMAGE_DIRECTORY))[1]
  
  for folder_name in folders:
    print(f'loading {folder_name}')
    file_names = next(os.walk(os.path.join(IMAGE_DIRECTORY, folder_name)))[2]
    for i in range(number_of_images):
      image_name = choice(file_names)
      image_path=os.path.join(IMAGE_DIRECTORY, folder_name, image_name)
      #print(image_path)
      label = one_hot_encode(folder_name)
      class_name =folder_name
      if "DS_Store" not in  image_path:
        img = Image.open(image_path)
        img = img.convert('L')
        img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
        train_data.append([np.array(img), label, class_name])
  if(shuffle_data == True):
    shuffle(train_data)
  print(f'we loaded, {len(train_data)}, images in the training dataset')
  return train_data